In [149]:
import pandas as pd
import numpy as np
%config Completer.use_jedi = False
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
host = '52.90.189.105'
region = 'us-east-1'

import os
import sys

from elasticsearch import helpers
import pandas as pd
import json
import unicodedata

In [150]:
import elasticsearch
#import streamlit as st
elasticsearch.__version__

(7, 12, 0)

In [276]:

def connect_elasticsearch():
    es = None
    es = Elasticsearch([{'host': host, 'port': 9200}])
    if es.ping():
        print('Yupiee  Connected ')
    else:
        print('Awww it could not connect!')
    return es
es = connect_elasticsearch()

2021-04-30 23:24:17.663 INFO    elasticsearch: HEAD http://52.90.189.105:9200/ [status:200 request:0.065s]


Yupiee  Connected 


In [306]:
df = pd.read_csv('elastic.csv')

In [307]:
df.columns

Index(['article_id', 'article_url', 'title', 'subtitle', 'claps', 'responses',
       'reading_time', 'tag', 'date', 'pro_title'],
      dtype='object')

In [308]:
df = df.drop(['article_id','pro_title', 'date','responses'], axis = 1)

In [292]:
def title_processing(text):
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    text = unicodedata.normalize("NFKD", text)
    return text

In [303]:
s = '<strong class="markup--strong markup--h4-strong">Why data science is important nowadays ?</strong>'


In [304]:
title_processing(s)

'Why data science is important nowadays ?'

In [309]:
df.loc[:,'subtitle']= df.loc[:,'subtitle'].apply(title_processing)

In [310]:
def generator_1(data):
    for c, line in enumerate(data):
        yield {
            "_index":"mediumarticle",
            "_type":"doc",
            "_id": c,
            "_soucre":{
                "title": line.get("title",""),
                "subtitle": line.get("subtitle",""),
                "tag":line.get("tag",""),
                "reading_time": line.get("reading_time",""),
                "article_url": line.get("article_url", ""),
                "claps": line.get("claps", "")
            }
        }
    raise StopIteration

In [311]:
data = df.to_dict('reocrds')
data = generator_1(data)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [312]:
first = next(data)


In [313]:
first

{'_index': 'mediumarticle',
 '_type': 'doc',
 '_id': 0,
 '_soucre': {'title': 'Calculating String Similarity in Python',
  'subtitle': 'Learn how to group together similar strings, and also how to...',
  'tag': 'Machine Learning',
  'reading_time': 7,
  'article_url': 'https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a',
  'claps': 605}}

In [314]:
es.index(index = 'mediumarticle', doc_type= 'doc', body = first, id = 0)


2021-04-30 23:33:25.215 INFO    elasticsearch: PUT http://52.90.189.105:9200/mediumarticle/doc/0 [status:200 request:0.029s]


{'_index': 'mediumarticle',
 '_type': 'doc',
 '_id': '0',
 '_version': 3,
 'created': False}

In [315]:
try:
    res = helpers.bulk(es,data)
    print('working')
except Exception as e:
    pass

2021-04-30 23:33:27.225 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.212s]
2021-04-30 23:33:27.385 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.146s]
2021-04-30 23:33:27.540 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.143s]
2021-04-30 23:33:27.689 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.135s]
2021-04-30 23:33:27.831 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.130s]
2021-04-30 23:33:28.018 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.175s]
2021-04-30 23:33:28.165 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.135s]
2021-04-30 23:33:28.314 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:0.137s]
2021-04-30 23:33:28.506 INFO    elasticsearch: POST http://52.90.189.105:9200/_bulk [status:200 request:

In [289]:
es.search(index = 'mediumarticle', doc_type= 'doc', body = { 'sort':{"claps":{"order":"desc"}},'size':2,'query':{'match':{"title": "calculating"}}})

2021-04-30 23:26:26.460 INFO    elasticsearch: POST http://52.90.189.105:9200/mediumarticle/doc/_search [status:200 request:0.033s]


{'took': 3,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'failed': 0},
 'hits': {'total': 1,
  'max_score': None,
  'hits': [{'_index': 'mediumarticle',
    '_type': 'doc',
    '_id': '0',
    '_score': None,
    '_source': {'_index': 'mediumarticle',
     '_type': 'doc',
     '_id': 0,
     '_soucre': {'title': 'Calculating String Similarity in Python',
      'subtitle': 'Learn how to group together similar strings, and also how\xa0to…',
      'tag': 'Machine Learning',
      'reading_time': 7,
      'article_url': 'https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a',
      'claps': 605}},
    'sort': [605]}]}}

In [240]:
def query_func(userinput):
    articles = []
    results = es.search(index = 'mediumarticle', doc_type= 'doc', body = { 'size':5,'query':{'match':{"title": userinput}}})
    hits = results['hits']['hits']
        
    for i in range(len(hits)):
        articles.append(hits[i]['_source']['_soucre'])
       
    return articles

In [316]:
es.search(index = 'mediumarticle', doc_type= 'doc', body = { 'size':5,'query':{'match':{"title": 'data science'}}})


2021-04-30 23:34:01.650 INFO    elasticsearch: POST http://52.90.189.105:9200/mediumarticle/doc/_search [status:200 request:0.034s]


{'took': 3,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'failed': 0},
 'hits': {'total': 604,
  'max_score': 3.816065,
  'hits': [{'_index': 'mediumarticle',
    '_type': 'doc',
    '_id': '13075',
    '_score': 3.816065,
    '_source': {'_soucre': {'title': 'Data Science Vs Data Analysis',
      'subtitle': 'The Difference Between Data Science and Data Analysis.',
      'tag': 'Machine Learning',
      'reading_time': 3,
      'article_url': 'https://medium.com/badatya-blog/data-science-vs-data-analysis-7652f9f981ac',
      'claps': 8}}},
   {'_index': 'mediumarticle',
    '_type': 'doc',
    '_id': '19059',
    '_score': 3.816065,
    '_source': {'_soucre': {'title': 'Data science without any data ',
      'subtitle': 'Why it’s important to hire data engineers early',
      'tag': 'Business',
      'reading_time': 6,
      'article_url': 'https://towardsdatascience.com/data-science-without-any-data-6c1ae9509d92',
      'claps': 1100}}},
   {'_index': 'mediumarticle

In [241]:
query_func('python')

2021-04-30 21:41:01.625 INFO    elasticsearch: POST http://52.90.189.105:9200/mediumarticle/doc/_search [status:200 request:0.032s]


[{'title': 'Python Everyday',
  'subtitle': ' strong class markup strong markup h strong code optimization interview and more strong ',
  'tag': 'Machine Learning',
  'reading_time': 7,
  'article_url': 'https://towardsdatascience.com/python-everyday-fa8946524532',
  'claps': 288},
 {'title': 'TURF Analysis in Python',
  'subtitle': 'a simple tool for statistical analysis',
  'tag': 'Marketing',
  'reading_time': 3,
  'article_url': 'https://medium.com/analytics-vidhya/turf-analysis-in-python-d8368f06c050',
  'claps': 5},
 {'title': 'Python Programming: Day 1',
  'subtitle': 'coming soon ',
  'tag': 'Machine Learning',
  'reading_time': 5,
  'article_url': 'https://medium.com/@rsaimurali/python-programming-day-1-bd57bceb2030',
  'claps': 0},
 {'title': 'DECISION TREES IN PYTHON',
  'subtitle': 'decision tree is a common machine learning algorithm used to ',
  'tag': 'Machine Learning',
  'reading_time': 3,
  'article_url': 'https://medium.com/@aadish-jain-88204/decision-trees-in-python